In [ ]:
#Gpu Connection Check
import tensorflow as tf
device_name=tf.test.gpu_device_name()
if device_name!='/device:GPU:0':
  raise SystemError('Gpu Not Found')
print('found gpu', device_name)

found gpu /device:GPU:0


In [ ]:
#importing libraries

import math
import numpy as np
import tensorflow as tf
import keras
import os
import collections
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, Dense, GRU,Bidirectional,RepeatVector,TimeDistributed,LSTM
from keras.models import Model,Sequential
from keras.optimizers import Adam,RMSprop

In [ ]:
#reading english sentecnes files from the drive
with open("/content/drive/My Drive/pruned_train.en",'r') as file:
  english_data=file.read()
english_data=english_data.lower()
english_list=english_data.split('\n')
file.close()


In [ ]:
#printing first 10 sentences
i=0
while i<10:
  print(english_list[i])
  i+=1

we generally find this capacity being directed towards mundane objects and close relations like wives , children and friends . 
i ' m sure i ' ll make a careless mistake
display web browser help
it is out of his mercy that he has made for you night and day , - that ye may rest therein , and that ye may seek of his grace ; - and in order that ye may be grateful . 
the disease caused by nutritional deficiency of ascorbic acid , formaly occuring in people who had not been to sea results in circular spots , stripes or patches scattered over thighs , arms , and trunk . 
the first temple was built by emperor ashoka in the 3rd century b . c . , and the present temple dates from the 5th or 6th centuries . 
so if you have a strategy , use it against me . 
severe pain in the chest , or shortness of breath . 
remember my choice for all links of this type . 
this effort needs a further push . 


In [ ]:
i=0
while i<10:
  print(hindi_list[i])
  i+=1

प्रायः हम इस भंडार को लौकिक विषयों पर और पत्नी , पुत्र मित्र आदि सगे संबंधियों पर प्रवर्तित कर देते हैं । 
मुझे जनता हूँ की मैं इस वीडियो में किसी भी समय
वेब ब्राउज़र मदद दिखाएँ
और उसने अपनी मेहरबानी से तुम्हारे वास्ते रात और दिन को बनाया ताकि तुम रात में आराम करो और दिन में उसके फज़ल व करम की तलाश करो और ताकि तुम लोग शुक्र करो
एब्सकार्बिक अम्ल कुपोषण की कमी के कारण द्वारा होने वाली बीमारी जो आमतौर से लोगो में होता है जिसके कारण गोलाकार धब्बे , पट्टियों या जांघों , बांह और धड़ पर चकत्ते । 
प्रथम मंदिर तीसरी शताब्दी बी . सी . में सम्राट अशोक द्वारा निर्मित कराया गया था और वर्तमान मंदिर पांचवीं या छठवीं शताब्दी में बनाए गए । 
तो अगर तुम्हें कोई दाँव करना हो तो आओ चल चुको
छाती में तेज दर्द अथवा सांस लेने में तकलीफ हो
इस प्रकार के सभी लिंक के लिए मेरा चयन याद रखें . 
इस प्रयास पर और अधिक जोर देने की आवश्यकता है । 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#reading hindi_sentecnes from the drive
with open("/content/drive/My Drive/pruned_train.hi",'r') as file:
  hindi_data=file.read()
hindi_list=hindi_data.split('\n')
file.close()

In [ ]:
#preprocessing data by removing the punctuation marks and special data
def remove_punctuation(txt):
  txt_output=""
  ty="'"
  ts="s"
  gh=0
  cnt=len(txt)
  for count,word in enumerate(txt):
    if gh is 1:
      gh=0
      continue
    if word in ty:
      if count+1<cnt and txt[count+1] in ts:
        gh=1
        continue 
    if word not in '''!"#$%&'()*+,-./:;”“<=>?@[\]^_`{|}~1234567890''':
      txt_output+=word
  return txt_output

In [ ]:
#punctuation marks removal from hindi dataset
cleaned_hindi_list=[]
for list in hindi_list:
  cleaned_hindi_list.append(remove_punctuation(list))

In [ ]:
#punctuation marks removal from english dataset
cleaned_english_list=[]
for list in english_list:
  cleaned_english_list.append(remove_punctuation(list))

In [ ]:
cleaned_hindi_list[5]

'प्रथम मंदिर तीसरी शताब्दी बी  सी  में सम्राट अशोक द्वारा निर्मित कराया गया था और वर्तमान मंदिर पांचवीं या छठवीं शताब्दी में बनाए गए । '

In [ ]:
cleaned_english_list[5]

'the first temple was built by emperor ashoka in the rd century b  c   and the present temple dates from the th or th centuries  '

In [ ]:
hindi_word_count=collections.Counter([word for list in cleaned_hindi_list for word in list.split()])
print(len(hindi_word_count))
print(hindi_word_count.most_common(3000))

231381
[('के', 581868), ('है', 530376), ('।', 452484), ('और', 432444), ('में', 401111), ('की', 343308), ('से', 280511), ('को', 253037), ('का', 221131), ('हैं', 160736), ('कि', 156099), ('पर', 135870), ('लिए', 131590), ('तो', 131442), ('एक', 128293), ('जो', 121558), ('नहीं', 119460), ('भी', 108145), ('हो', 103054), ('ही', 93896), ('किया', 85177), ('यह', 84895), ('ने', 82591), ('कर', 81471), ('वह', 78192), ('इस', 71234), ('करने', 68311), ('अपने', 65621), ('था', 61309), ('न', 60672), ('तुम', 58016), ('या', 57617), ('वे', 55524), ('कुछ', 54216), ('गया', 47851), ('कोई', 47196), ('उनके', 46946), ('तथा', 44659), ('थे', 43306), ('साथ', 42496), ('लोग', 42063), ('हम', 41873), ('लोगों', 41443), ('किसी', 40930), ('करते', 40446), ('जब', 38949), ('दिया', 38461), ('अल्लाह', 37732), ('रूप', 35715), ('फिर', 35229), ('उसके', 34581), ('ख़ुदा', 34100), ('ये', 33925), ('क्या', 32833), ('द्वारा', 32706), ('तक', 32609), ('अपनी', 31773), ('जाता', 31435), ('उन्हें', 30747), ('होता', 29404), ('जा', 29385), ('उस

In [ ]:
#list of the most commmon words in hindi that are to be removed
remove=['के','है','और','में','की','से','को','का','हैं','कि','पर','लिए','तो','एक','जो','1','2','।','3','4','5','6','7','8','9','0','१','२','३','४','५','६','७','८','९','०','नहीं','भी','हो','ही','किया','यह','ने','कर','वह','इस','करने','अपने','था','न','तुम','या','वे','कुछ','गया','कोई','उनके','तथा','थे','साथ','लोग','हम','लोगों','किसी','करते','जब','दिया','अल्लाह','रूप','फिर','उसके','ये','क्या','द्वारा','तक','अपनी','जाता','उन्हें','होता','जा','उसे','उन','उस','रहे','करता','थी','करना','सकता','मैं','हुए','करें','बहुत','दो','हमने','आप','तुम्हारे', 'रहा', 'समय','उनकी','पास','बात','बाद', 'यदि','तीसरी','तरह','सी','बी', 'पहले','हुआ','अधिक','कहा','वाले','ओर','उन्होंने','उसकी','होगा','होती','होने','गई','दी','चाहिए','गए','इसके','अगर','सभी','हमारे', 'ऐसे','इन','तुम्हें', 'करो', 'जाने','दे','ऐसा','कार्य','उसने','लेकिन','लिया','किए','अन्य','जिस','एवं','हुई', 'जाती', 'मुझे', 'ले', 'अब','होते', 'कम','उसका','सब', 'ताकि', 'व','उनका','वही', 'हर','बारे','वाला']

In [ ]:
with open('/content/final_stopwords.txt','r') as file:
  data=file.read()
remove_these_hindi_words=data.split('\n')
remove_these_hindi_words[-1]
remove_these_hindi_words+=remove

In [ ]:
english_word_count=collections.Counter([word for list in cleaned_english_list for word in list.split()])
print(len(english_word_count))
print(english_word_count.most_common(1000))

138721
[('the', 937228), ('of', 549806), ('and', 486515), ('to', 385983), ('in', 301120), ('a', 274682), ('is', 231405), ('for', 151334), ('that', 145826), ('you', 139005), ('it', 118978), ('not', 103996), ('be', 103745), ('he', 102195), ('they', 100380), ('with', 94155), ('are', 93527), ('on', 85557), ('as', 85523), ('this', 80848), ('have', 79783), ('was', 77262), ('from', 76165), ('by', 75411), ('we', 75310), ('them', 75300), ('his', 73451), ('or', 72172), ('will', 72033), ('who', 69951), ('which', 62702), ('their', 61769), ('has', 56628), ('but', 56277), ('i', 50280), ('an', 50220), ('all', 49849), ('allah', 49426), ('s', 44708), ('your', 43611), ('at', 43001), ('him', 41705), ('if', 40455), ('those', 39442), ('so', 39374), ('when', 39351), ('do', 38289), ('one', 38098), ('there', 36956), ('had', 35257), ('what', 34081), ('were', 33426), ('been', 31273), ('can', 30228), ('people', 30107), ('god', 29882), ('no', 29660), ('then', 28744), ('our', 27892), ('its', 27723), ('lord', 24539

In [ ]:
#list of the english words to be removed from the dataset
remove_these_english_words=['the','of','b','c','and','st','nd','rd','th','1','2','3','4','5','6','7','8','9','0','ll','m','i','to','in','a','is','for','that','you','it','not','be','he','they','with','are','on','as','this','have','was','from','by','we','them','his','or','will','who','which','their','has','but','i','an','all','allah','s','your','at','him','if','those','so','when','do','one','there','had','what','were','been','can','people','no','then','our','its','may','also','would','only','other','any','shall','some','these','my','day','should','out','more','into','made','said','t','than','such','over','most','could','did','even','under','being','because','upon','very','she','while','ye']

In [ ]:
i=0
while i<10:
  print(cleaned_english_list[i])
  i+=1

we generally find this capacity being directed towards mundane objects and close relations like wives  children and friends  
i  m sure i  ll make a careless mistake
display web browser help
it is out of his mercy that he has made for you night and day   that ye may rest therein  and that ye may seek of his grace   and in order that ye may be grateful  
the disease caused by nutritional deficiency of ascorbic acid  formaly occuring in people who had not been to sea results in circular spots  stripes or patches scattered over thighs  arms  and trunk  
the first temple was built by emperor ashoka in the rd century b  c   and the present temple dates from the th or th centuries  
so if you have a strategy  use it against me  
severe pain in the chest  or shortness of breath  
remember my choice for all links of this type  
this effort needs a further push  


In [ ]:
i=0
for list in cleaned_english_list:
  for word in list.split():
    print(word)
  print()
  i+=1
  if(i>10):
    break

we
generally
find
this
capacity
being
directed
towards
mundane
objects
and
close
relations
like
wives
children
and
friends

i
m
sure
i
ll
make
a
careless
mistake

display
web
browser
help

it
is
out
of
his
mercy
that
he
has
made
for
you
night
and
day
that
ye
may
rest
therein
and
that
ye
may
seek
of
his
grace
and
in
order
that
ye
may
be
grateful

the
disease
caused
by
nutritional
deficiency
of
ascorbic
acid
formaly
occuring
in
people
who
had
not
been
to
sea
results
in
circular
spots
stripes
or
patches
scattered
over
thighs
arms
and
trunk

the
first
temple
was
built
by
emperor
ashoka
in
the
rd
century
b
c
and
the
present
temple
dates
from
the
th
or
th
centuries

so
if
you
have
a
strategy
use
it
against
me

severe
pain
in
the
chest
or
shortness
of
breath

remember
my
choice
for
all
links
of
this
type

this
effort
needs
a
further
push

the
books
of
abraham
and
moses



In [ ]:
#removing words from english dataset
final_english_list=[]
for list in cleaned_english_list:
  txt=''
  for word in list.split():
    if word not in remove_these_english_words:
      txt+=word+' '
  final_english_list.append(txt)

In [ ]:
final_english_list[1]

'sure make careless mistake '

In [ ]:
i=0
for list in final_english_list:
  for word in list.split():
    print(word)
  print()
  i+=1
  if(i>10):
    break

generally
find
capacity
directed
towards
mundane
objects
close
relations
like
wives
children
friends

sure
make
careless
mistake

display
web
browser
help

mercy
night
rest
therein
seek
grace
order
grateful

disease
caused
nutritional
deficiency
ascorbic
acid
formaly
occuring
sea
results
circular
spots
stripes
patches
scattered
thighs
arms
trunk

first
temple
built
emperor
ashoka
century
present
temple
dates
centuries

strategy
use
against
me

severe
pain
chest
shortness
breath

remember
choice
links
type

effort
needs
further
push

books
abraham
moses



In [ ]:
i=0
while i<10:
  print(final_english_list[i])
  i+=1

generally find capacity directed towards mundane objects close relations like wives children friends 
sure make careless mistake 
display web browser help 
mercy night rest therein seek grace order grateful 
disease caused nutritional deficiency ascorbic acid formaly occuring sea results circular spots stripes patches scattered thighs arms trunk 
first temple built emperor ashoka century present temple dates centuries 
strategy use against me 
severe pain chest shortness breath 
remember choice links type 
effort needs further push 


In [ ]:
i=0
for list in cleaned_hindi_list:
  for word in list.split():
    print(word)
  print()
  i+=1
  if(i>10):
    break

प्रायः
हम
इस
भंडार
को
लौकिक
विषयों
पर
और
पत्नी
पुत्र
मित्र
आदि
सगे
संबंधियों
पर
प्रवर्तित
कर
देते
हैं
।

मुझे
जनता
हूँ
की
मैं
इस
वीडियो
में
किसी
भी
समय

वेब
ब्राउज़र
मदद
दिखाएँ

और
उसने
अपनी
मेहरबानी
से
तुम्हारे
वास्ते
रात
और
दिन
को
बनाया
ताकि
तुम
रात
में
आराम
करो
और
दिन
में
उसके
फज़ल
व
करम
की
तलाश
करो
और
ताकि
तुम
लोग
शुक्र
करो

एब्सकार्बिक
अम्ल
कुपोषण
की
कमी
के
कारण
द्वारा
होने
वाली
बीमारी
जो
आमतौर
से
लोगो
में
होता
है
जिसके
कारण
गोलाकार
धब्बे
पट्टियों
या
जांघों
बांह
और
धड़
पर
चकत्ते
।

प्रथम
मंदिर
तीसरी
शताब्दी
बी
सी
में
सम्राट
अशोक
द्वारा
निर्मित
कराया
गया
था
और
वर्तमान
मंदिर
पांचवीं
या
छठवीं
शताब्दी
में
बनाए
गए
।

तो
अगर
तुम्हें
कोई
दाँव
करना
हो
तो
आओ
चल
चुको

छाती
में
तेज
दर्द
अथवा
सांस
लेने
में
तकलीफ
हो

इस
प्रकार
के
सभी
लिंक
के
लिए
मेरा
चयन
याद
रखें

इस
प्रयास
पर
और
अधिक
जोर
देने
की
आवश्यकता
है
।

इबराहीम
और
मूसा
के
सहीफ़ों
में
भी
है



In [ ]:
#removing words from the hindi list
final_hindi_list=[]
for list in cleaned_hindi_list:
  txt=''
  for word in list.split():
    if word not in remove_these_hindi_words:
      txt+=word+' '
  final_hindi_list.append(txt)

In [ ]:
final_hindi_list[650456]

'विवाहिता असाध्य रोग ग्रस्त कारण जलवायु परिवर्तन कहीं कहीं करती '

In [ ]:
i=0
for list in final_hindi_list:
  for word in list.split():
    print(word)
  print()
  i+=1
  if(i>10):
    break

प्रायः
भंडार
लौकिक
विषयों
पत्नी
पुत्र
मित्र
सगे
संबंधियों
प्रवर्तित
देते

जनता
वीडियो

वेब
ब्राउज़र
मदद
दिखाएँ

मेहरबानी
वास्ते
रात
दिन
बनाया
रात
आराम
दिन
फज़ल
करम
तलाश
शुक्र

एब्सकार्बिक
अम्ल
कुपोषण
कमी
कारण
वाली
बीमारी
आमतौर
लोगो
जिसके
कारण
गोलाकार
धब्बे
पट्टियों
जांघों
बांह
धड़
चकत्ते

प्रथम
मंदिर
शताब्दी
सम्राट
अशोक
निर्मित
कराया
वर्तमान
मंदिर
पांचवीं
छठवीं
शताब्दी
बनाए

दाँव
आओ
चल
चुको

छाती
तेज
दर्द
सांस
लेने
तकलीफ

प्रकार
लिंक
चयन
याद

प्रयास
जोर
देने
आवश्यकता

इबराहीम
मूसा
सहीफ़ों



In [ ]:
i=0
while i<10:
  print(final_hindi_list[i])
  i+=1

प्रायः भंडार लौकिक विषयों पत्नी पुत्र मित्र सगे संबंधियों प्रवर्तित देते 
जनता वीडियो 
वेब ब्राउज़र मदद दिखाएँ 
मेहरबानी वास्ते रात दिन बनाया रात आराम दिन फज़ल करम तलाश शुक्र 
एब्सकार्बिक अम्ल कुपोषण कमी कारण वाली बीमारी आमतौर लोगो जिसके कारण गोलाकार धब्बे पट्टियों जांघों बांह धड़ चकत्ते 
प्रथम मंदिर शताब्दी सम्राट अशोक निर्मित कराया वर्तमान मंदिर पांचवीं छठवीं शताब्दी बनाए 
दाँव आओ चल चुको 
छाती तेज दर्द सांस लेने तकलीफ 
प्रकार लिंक चयन याद 
प्रयास जोर देने आवश्यकता 


In [ ]:
#function for reversing the list
def reversing(txt):
  txt2=[]
  txt2 = txt[::-1]
  return txt2

In [ ]:
num_words=25000

In [ ]:
#adding starting and ending position in each english sentence
start_position="start "
end_position=" endlj"
print(end_position)

 endlj


In [ ]:
final_english_list3=[]
for list in final_english_list:
  final_list=start_position+list+end_position
  final_english_list3.append(final_list)  

In [ ]:
final_english_list3[8]

'start remember choice links type  endlj'

In [ ]:
final_english_list4=final_english_list3[0:100000]
final_hindi_list4=final_hindi_list[0:100000]

In [ ]:
#making Tokenizing class having Keras Tokenization class with some additional functions 
class Tokenizing_Text(Tokenizer):
  def __init__(self,texts,padding,reverse=False,num_words=None):
    #Inheritance of the Tokenizer class from keras 
    Tokenizer.__init__(self,num_words=num_words,char_level=False)

    self.fit_on_texts(texts)

    self.index_to_words=dict(zip(self.word_index.values(),
                                 self.word_index.keys()))
    self.tokens=self.texts_to_sequences(texts)

    if reverse:
      for count,g in enumerate(self.tokens,0):
        self.tokens[count]=reversing(g)
      truncating='pre'
    
    else:
      truncating='post'
    
    self.num_tokens = [len(g) for g in self.tokens]
    self.max_tokens = 24
    
    self.tokens_padded=pad_sequences(self.tokens,
                                     maxlen=self.max_tokens,
                                     padding=padding,
                                     truncating=truncating)
    
#function for converting tokens of the language to words
  def token_to_word(self,token):
      word = " " if token == 0 else self.index_to_words[token]
      return word
# function for making sentence out of gicen tokens of the words
  def tokens_to_string(self,tokens):
      words=[self.index_to_words[token] for token in tokens if token!=0]
      text = " ".join(words)

      return text
    
#function for converting the text to the tokens 
  def text_to_tokens(self, text, reverse=False, padding=False):

      tokens=self.texts_to_sequences([text])
      tokens=np.array(tokens)

      if reverse:

        tokens=np.flip(tokens,axis=1)
        truncating='pre'

      else:
        truncating='post'

      if padding:
        tokens=pad_sequences(tokens,maxlen=self.max_tokens,padding='pre',truncating=truncating)

        return tokens 

    

In [ ]:
english_tokenized2=Tokenizing_Text(texts=final_english_list4,padding='post',reverse=False,num_words=num_words)

In [ ]:
hindi_tokenized2=Tokenizing_Text(texts=final_hindi_list4,padding='pre',reverse=True,num_words=num_words)

In [ ]:
hindi_tokens2=hindi_tokenized2.tokens_padded
english_tokens2=english_tokenized2.tokens_padded
print(hindi_tokens2.shape)
print(english_tokens2.shape)

(100000, 24)
(100000, 24)


In [ ]:
starting_token2=english_tokenized2.word_index[start_position.strip()]
print(starting_token2)

1


In [ ]:
hindi_tokens2[1]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0, 828, 321], dtype=int32)

In [ ]:
english_tokens2[1]

array([    1,   482,    20, 11493,  2984,     2,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0], dtype=int32)

In [ ]:
ending_token2=english_tokenized2.word_index[end_position.strip()]

In [ ]:
print(ending_token2)

2


In [ ]:
english_tokenized2.tokens_to_string(english_tokens2[45])

'start soon found lion sleeping nearby honey paws endlj'

In [ ]:
hindi_tokenized2.tokens_to_string(hindi_tokens2[45])

'पड़ दिखाई शेर सोया पड़ोस जल्द'

In [ ]:
# making the encoder decoder model

In [ ]:
encoder_input_data=hindi_tokens2
hindi_tokens2.shape

(100000, 24)

In [ ]:
decoder_input_data=english_tokens2[:,:-1]
decoder_output_data=english_tokens2[:,1:]

In [ ]:
decoder_output_data.shape

(100000, 23)

In [ ]:
decoder_output_data=np.reshape(decoder_output_data,(100000,23,1))
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_output_data.shape)

(100000, 24)
(100000, 23)
(100000, 23, 1)


In [ ]:
encoder_input = Input(shape=(None, ),name="encoder_input")

In [ ]:
embedding_size=128

In [ ]:
# defining Embedding layer for encoder input
encoder_embedding=Embedding(input_dim=num_words,
                            output_dim=embedding_size,
                            name='encoder_embedding')


In [ ]:
state_size=512

In [ ]:
# making 3 layers of Gated Recurrent Unit
encoder_gru1 = GRU(state_size, name='encoder_gru1',
                   return_sequences=True)
encoder_gru2 = GRU(state_size, name='encoder_gru2',
                   return_sequences=True)
# last layer will not return the sequence as the single vector will be fed as initial state into the decoder 
encoder_gru3 = GRU(state_size, name='encoder_gru3',
                   return_sequences=False)

In [ ]:
#connecting various layers of the encoder
def connect_encoder():
  net=encoder_input
  net=encoder_embedding(net)
  net=encoder_gru1(net)
  net=encoder_gru2(net)
  net=encoder_gru3(net)

  encoder_output=net
  return encoder_output

In [ ]:
encoder_output=connect_encoder()

In [ ]:
decoder_initial_state = Input(shape=(state_size,),
                              name='decoder_initial_state')

In [ ]:
decoder_input = Input(shape=(None, ), name='decoder_input')

In [ ]:
#making the embedding layer for decoder input
decoder_embedding = Embedding(input_dim=num_words,
                              output_dim=embedding_size,
                              name='decoder_embedding')

In [ ]:
decoder_gru1 = GRU(state_size, name='decoder_gru1',
                   return_sequences=True)
decoder_gru2 = GRU(state_size, name='decoder_gru2',
                   return_sequences=True)
decoder_gru3 = GRU(state_size, name='decoder_gru3',
                   return_sequences=True)

In [ ]:
decoder_dense = Dense(num_words,
                      activation='softmax',
                      name='decoder_output')

In [ ]:
def connect_decoder(initial_state):
    net = decoder_input
    net = decoder_embedding(net)
    net = decoder_gru1(net, initial_state=initial_state)
    net = decoder_gru2(net, initial_state=initial_state)
    net = decoder_gru3(net, initial_state=initial_state)

    decoder_output = decoder_dense(net)
    return decoder_output

In [ ]:

decoder_output = connect_decoder(initial_state=encoder_output)

In [ ]:
model_train = Model(inputs=[encoder_input, decoder_input],
                    outputs=[decoder_output])

In [ ]:
model_encoder = Model(inputs=[encoder_input],
                      outputs=[encoder_output])

In [ ]:

decoder_output = connect_decoder(initial_state=decoder_initial_state)

model_decoder = Model(inputs=[decoder_input, decoder_initial_state],
                      outputs=[decoder_output])

In [ ]:
from keras.optimizers import RMSprop
model_train.compile(optimizer=RMSprop(lr=0.003),
                    loss='sparse_categorical_crossentropy')

In [ ]:

x_data = \
{
    'encoder_input': encoder_input_data,
    'decoder_input': decoder_input_data
}

In [ ]:

y_data = \
{
    'decoder_output': decoder_output_data
}

In [ ]:
#training our model with 25 epochs and validation set 10%
model_train.fit(x=x_data,
                y=y_data,
                batch_size=512,
                epochs=25,
                validation_split=0.1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 90000 samples, validate on 10000 samples
Epoch 1/25
90000/90000 [==============================] - 117s 1ms/step - loss: 4.0885 - val_loss: 3.4536
Epoch 2/25
90000/90000 [==============================] - 115s 1ms/step - loss: 3.4358 - val_loss: 3.3179
Epoch 3/25
90000/90000 [==============================] - 115s 1ms/step - loss: 3.2950 - val_loss: 3.3097
Epoch 4/25
90000/90000 [==============================] - 115s 1ms/step - loss: 3.2361 - val_loss: 3.2665
Epoch 5/25
90000/90000 [==============================] - 115s 1ms/step - loss: 3.1757 - val_loss: 3.2282
Epoch 6/25
90000/90000 [==============================] - 115s 1ms/step - loss: 3.1167 - val_loss: 3.2107
Epoch 7/25
90000/90000 [==============================] - 115s 1ms/step - loss: 3.0614 - val_loss: 3.1835
Epoch 8/25
90000/90000 [==============================] - 115s 1ms/step - loss: 3.0091 - val_loss: 3.1804
Epoch 9/25
90000/90000 [==============================] - 114s 1ms/step - loss: 2.9413 - val_loss: 3.1

In [ ]:
#saving the model
model_train.save('model_train.h5')

In [ ]:
#function for translating the text where input is hindi sentences and predicted and actual english sentences are shown
def translating(i_text, true_o_text=None):

    i_tokens = hindi_tokenized2.text_to_tokens(text=i_text,
                                                reverse=True,
                                                padding=True)
    
 
    initial_state = model_encoder.predict(i_tokens)


    max_tokens = english_tokenized2.max_tokens

    print(max_tokens)
    shape = (1, max_tokens)
    decoder_input_data = np.zeros(shape=shape, dtype=np.int)

 
    token_int = starting_token2


    o_text = ''

    count_tokens = 0
    while token_int != ending_token2 and count_tokens < max_tokens:
        decoder_input_data[0, count_tokens] = token_int
        x_data = \
        {
            'decoder_initial_state': initial_state,
            'decoder_input': decoder_input_data
        }

        decoder_output = model_decoder.predict(x_data)

        token_onehot = decoder_output[0, count_tokens, :]  
        token_int = np.argmax(token_onehot)
        sampled_word = english_tokenized2.token_to_word(token_int)
        if sampled_word is "endj":
          break
        o_text += " " + sampled_word
        count_tokens += 1

    output_tokens = decoder_input_data[0]
    
    print("Input text:")
    print(i_text)
    print()

    
    print("Translated text:")
    print(o_text)
    print()


    if true_o_text is not None:
        print("True output text:")
        print(true_o_text)
        print()
    return o_text,true_o_text

In [ ]:
#making reference and hypothesis for the blue score calculation
def making_reference_hypothesis(txt):
  txt_output=[]
  for words in txt.split():
    txt_output.append(words)
  return txt_output

In [ ]:
translating(i_text=final_hindi_list4[5],true_o_text=final_english_list4[5])

24
Input text:
प्रथम मंदिर शताब्दी सम्राट अशोक निर्मित कराया वर्तमान मंदिर पांचवीं छठवीं शताब्दी बनाए 

Translated text:
 north east south south bengal south himalaya specially himalaya south himalaya tiger endlj

True output text:
start first temple built emperor ashoka century present temple dates centuries  endlj



In [ ]:
hypothesis = making_reference_hypothesis('first century built temple built built temple built built present century endlj')
reference = making_reference_hypothesis('start first temple built emperor ashoka century present temple dates centuries  endlj')
#there may be several references
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
print(BLEUscore)

NameError: ignored

In [ ]:
translating(i_text=final_hindi_list4[8967],true_o_text=final_english_list4[8967])

24
Input text:
बस हमारी पहली मृत्यु दोबारा उठाए जानेवाले 

Translated text:
 see die sun mocked endlj

True output text:
start first death never raised again  endlj



In [ ]:
translating(i_text='मृत्यु',true_o_text='die')

24
Input text:
मृत्यु

Translated text:
 wife die endlj

True output text:
die



In [ ]:
final_hindi_list[356234]

'हमे उत्तर मिश्रित संख्या लिखने '

In [ ]:
translating(i_text="फ़िरऔन सेना पीछा अन्ततः पानी उनपर छा जैसाकि उनपर छा जाना",true_o_text=final_english_list4[35623])

24
Input text:
फ़िरऔन सेना पीछा अन्ततः पानी उनपर छा जैसाकि उनपर छा जाना

Translated text:
 fir awn pursued offerings pursued bar sound endlj

True output text:
start pharaoh pursued troops whereat engulfed engulfed sea  endlj



In [ ]:
hypothesis = making_reference_hypothesis('pharaoh pursued army sea pursued sea endlj')
reference = making_reference_hypothesis('pursued troops whereat engulfed engulfed sea  endlj')
#there may be several references
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
print(BLEUscore)

0.5169731539571706


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
translating(i_text=final_hindi_list[3402],true_o_text=final_english_list4[3402])

24
Input text:
आकाशों धरती प्रकाश प्रकाश मिसाल ऐसी ताक़ चिराग़ चिराग़ फ़ानूस फ़ानूस चमकता तारा चिराग़ ज़ैतून बरकतवाले वृक्ष तेल जलाया पूर्वी पश्चिमी तेल भड़का पड़ता यद्यपि आग छुए प्रकाश प्रकाश चाहता प्रकाश प्राप्त मार्ग दिखा देता मिशालें प्रस्तुत चीज़ जानता 

Translated text:
 light heavens earth light lamp lamp lamp lamp lamp lamp lamp lamp lamp lamp lamp lamp lamp lamp lamp lamp lamp lamp lamp lamp

True output text:
start light heavens earth light likened lamp niche lamp glass shade glass shade like glittering star lamp lit olive oil blessed tree neither eastern nor western oil going shine forth itself though fire touched light light guides light whomever wills cites parables make message clear perfect knowledge everything  endlj



(' light heavens earth light lamp lamp lamp lamp lamp lamp lamp lamp lamp lamp lamp lamp lamp lamp lamp lamp lamp lamp lamp lamp',
 'start light heavens earth light likened lamp niche lamp glass shade glass shade like glittering star lamp lit olive oil blessed tree neither eastern nor western oil going shine forth itself though fire touched light light guides light whomever wills cites parables make message clear perfect knowledge everything  endlj')

In [ ]:
translating(i_text=final_hindi_list[3400],true_o_text=final_english_list4[3400])

24
Input text:
पुत्र कमरूत्रीन सात वर्ष इंग़्लैड वहां भारतीय मूल सॉलिसिटर 

Translated text:
 mother mother roman busy indian esteem bharati puran proud endlj

True output text:
start another son camruddin stayed england seven years first indian become solicitor  endlj



In [ ]:
translating(i_text=final_hindi_list[2500],true_o_text=final_english_list4[2500])

24
Input text:
केवल लोमड़ी लाख दूसरी लोमडियों मैंने मित्र बना विश्व भर अनोखी 

Translated text:
 last year drought drought drought roads roads factories endlj

True output text:
start fox like hundred thousand foxes friend now unique world  endlj



In [ ]:
hypothesis = making_reference_hypothesis('mixed mixed number mixed number endlj')
reference = making_reference_hypothesis('start mixed number simplify write answer  endlj')
#there may be several references
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
print(BLEUscore)

0.4760116549244004


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
translating(i_text=final_hindi_list[23],true_o_text=final_english_list[23])

24
Input text:
ख़ुदा बारगाह अर्ज़ ऐ पालने तूने नाज़िल उसपर ईमान लाए तेरे रसूल पैरवी इख्तेयार पस तू हमें गवाहों दफ्तर लिख 

Translated text:
 plotted wing say moses revealed unto wills revealed me unto – hell drowned safely warn earned endlj

True output text:
plotted devised best devisers 



In [ ]:
translating(i_text=final_hindi_list[232],true_o_text=final_english_list[232])

24
Input text:
लोक दंड भय परलोक यंत्रणाओं 

Translated text:
 gave wealth purified unjustly purified endlj

True output text:
certainly less powerful restraint fear punishments world fear torments next 



In [ ]:
import nltk

In [ ]:
bleu_s=[]
for count,list in enumerate(final_hindi_list4[300:700],300):
  o_text,t_o_text=translating(list,final_english_list4[count])
  hypothesis = making_reference_hypothesis(o_text)
  reference = making_reference_hypothesis(t_o_text)
  BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
  bleu_s.append(BLEUscore)
  print(BLEUscore)

24
Input text:
कांगड़ा पहाड़ी इलाके जालंधर मैदानों कटे कटोच सरदार पहाड़ों सीमित 

Translated text:
 fort hills ji island elephant up endlj

True output text:
start hilly areas kangra cut off plains jaland hara katoch chieftains confined themselves hills  endlj

0.23315591526465868
24
Input text:
जाना जिन्होंने भुला आपको भूल बैठे अवज्ञाकारी 

Translated text:
 say promised easy us endlj

True output text:
start like forgot god caused forget own souls rebellious ones  endlj

0.20142070913129906


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


24
Input text:
नेस्ट बॉक्स तल सेंटीमीटर छेद डिजाइन छोटे खरगोशों नेस्ट बॉक्सत आने रोकता 

Translated text:
 four four months band structure genes meters meters car holding endlj

True output text:
start designing hole bottom nest box prevents young rabbits coming nest box  endlj

0.4578141331660858
24
Input text:
एज़रा पाउंड जिन्होंने कवि देखे भाव प्रतिमा ठीक समझने कोशिश आवाज अकेली पड़ 

Translated text:
 next poem poem noted tells up story story written endlj

True output text:
start ezra pound saw poet sees another tried correct distorted image lone voice  endlj

0.3769486629893372
24
Input text:
शोरा अम्ल एक्वा फोर्टीस 

Translated text:
 eggs acid endlj

True output text:
start nitric acid called aqua fortis  endlj

0.23818690214719165
24
Input text:
माध्यम लूप 

Translated text:
 apply render endlj

True output text:
start falls way through loop done  endlj

0.20029051217596075
24
Input text:
वाणिज्यिक बैंकों निश्चित समाजिक दायित्वों उद्देश्यों आर्थिक विकास मुख्य धारा लाने दृष्टि स

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Input text:
जिसने हमें उदार अनुग्रह रहने घर उतारा हमें मशक़्क़त उठानी पड़ती हमें थकान आती 

Translated text:
 say promised initiate promised repeat good repeat habits divinity endlj

True output text:
start grace hath installed us mansion eternity where toil toucheth us nor weariness affect us  endlj

0.3086194627209984
24
Input text:
बस पैसे लागत आसपास बैठे 

Translated text:
 need make something something putting bit sphere endlj

True output text:
start sitting around just costing money  endlj

0.5946035575013605
24
Input text:
ध्येय वजह वर्षो अनुकूल बनने यथाशक्ति प्रयत्न 

Translated text:
 bharati prepared psychological sacrifice sacrifice endlj

True output text:
start functioned together during years utmost adapt ourselves each  endlj

0.3280443278078623
24
Input text:
पैसों बिल्कुल मोह पैसे रखे 

Translated text:
 person luggage money endlj

True output text:
start never loved money nor ever keep money her  endlj

0.18762935180380186
24
Input text:
मुकदमा हस्ताक्षरित सत्यापित व

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Input text:
सबब ख़ुदा आयतों हँसी ठट्ठा बना रखा दुनयावी ज़िन्दगी तुमको धोखे डाल ग़रज़ दुनिया निकाले जाएँगे मौका जाएगा राज़ी 

Translated text:
 say commits evil deeds repaid good deeds good deeds deeds deeds deeds dwell dwell good good good good deeds utter dwell dwell dwell dwell

True output text:
start took verses ridicule worldly life deluded removed nor asked appease  endlj

0
24
Input text:
तीसरा पक्ष हमें बालकनी मदद 

Translated text:
 helped known comic considered script endlj

True output text:
start third side helps us go balcony  endlj

0.45782273986766686
24
Input text:
उत्तर बंगाल हुदुम देव नृत्य पानी बरसाने 

Translated text:
 yamuna paddy sur used tourist yamuna endlj

True output text:
start hudum deo dance north bengal rain inducing magical dance  endlj

0.34718201116725705
24
Input text:
भली भाँति जानते कहेंगे सबसे अच्छी सम्मतिवाला कहेगा बस दिन ठहरे 

Translated text:
 know know say wills say wills say shalt knowledge endlj

True output text:
start know best say faires

In [ ]:
add=0
cnt=0
for no in bleu_s:
  cnt+=1
  add+=no
avg_bleu_score=add/cnt
print(avg_bleu_score)

0.4332277695693803


In [ ]:
translating(i_text=final_hindi_list[342],true_o_text=final_english_list4[342])

NameError: ignored